# LLMs vs. TextWorld
This notebook tests and proves the hypothesis that modern, off-the-shelf LLMs — even non-reasoning models such as GPT-4o — can achieve high win rates in TextWorld when given a detailed prompt and a walkthrough of an example game, without any task-specific training. Games are generated via `tw-cooking --recipe 3 --take 2 --go 12 --open --cook --cut --drop`, essentially configuring them for maximum difficulty. Of the four models tested, the best-performing is Llama-3.1-405B-Instruct-FP8, which won at 96 out of 100 seeds on its first attempt, and all 100 within two attempts. Difficulty navigating the map is the most common source of failure.

## Environment setup

This section sets up a portable notebook environment which can run on a local kernel or on Google Colab. It pip-installs dependencies, sets up a place to store experimental results, and sets up a mechanism for accessing API keys.

In [3]:
import jupyter_core, os, os.path, sys, sysconfig

First we need to pip-install some prerequisites and locate the `tw-make` binary. `tw-make` generates the TextWorld games that we'll be testing against.

In [4]:
!{sys.executable} -m pip install textworld openai

TW_MAKE_BIN = os.path.join(sysconfig.get_path("scripts"), "tw-make")
assert os.path.exists(TW_MAKE_BIN)


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /home/dfranke/.local/share/pipx/venvs/jupyterlab/bin/python -m pip install --upgrade pip


Now we define `WORK_DIR`, where we'll store generated games and experimental results. We'll use a Google Drive if we're running in Colab, otherwise use the Jupyter data directory. You might want to customize this cell.

In [5]:
USE_GOOGLE_DRIVE : bool = True
OFFLINE_WORK_DIR : str = os.path.join(
    jupyter_core.paths.jupyter_data_dir(),
"textworld")
ONLINE_WORK_DIR : str = "/content/drive/MyDrive/textworld"

if USE_GOOGLE_DRIVE:
    try:
        from google.colab import drive
        drive.mount("/content/drive", force_remount=True)
        WORK_DIR : str = ONLINE_WORK_DIR
    except ImportError:
        WORK_DIR = OFFLINE_WORK_DIR
else:
    WORK_DIR = OFFLINE_WORK_DIR

This next cell defines a function for accessing API keys. It supports getting them from Colab's secret store or from environment variables. You may need to modify it if you want to access them through some other mechanism.

In [6]:
def get_api_key(service : str) -> str:
    try:
        from google.colab import userdata

        secret = userdata.get(service)
        if secret is not None:
            return secret
    except ImportError:
        pass
    secret = os.environ.get(service.upper())
    if secret is None:
        warnings.warn(
            "Failed to retrieve API key for {}. Set it in Colab secrets "
            "or via the {} environment variable.".format(
                service, service.upper()
            )
        )
    return secret

## Apparatus

In this section we'll develop the apparatus for generating TextWorld games, interfacing LLMs to them, and recording their performance.

In [7]:
import copy, hashlib, itertools, json, jupyter_core, multiprocessing, openai, os, \
    os.path, random, re, subprocess, tarfile, tempfile, textworld, textworld.gym, \
    time, urllib, warnings
from typing import Any, Dict, Iterable, List, Literal, Mapping, NotRequired, Optional, \
    Tuple, TypedDict, Union, cast

We support generating games using `tw-make` or downloading pre-generated games from the web. We'll generate all the games that we'll be testing the LLMs against, but for the first experiment we'll download an example game from the TextWorld website and use that to generate a walkthrough to include in the LLMs' prompt.

The constructor for `GeneratedGame` takes a seed and a list of arguments and invokes `tw-make` accordingly. It stores the game under `$WORK_DIR/games` and names the file after a SHA-256 hash of the arguments that were given. If the file already exists, it skips regenerating the game.

The constructor for `FetchedGame` takes a URL for the game's `.z8` file and optionally a second one for the associated `.json` file. The `.z8` file contains the game itself and the `.json` file contains some metadata that the TextWorld gym can optionally use to report more detailed information about the state of a game. The fetched game is stored in a temporary directory which is cleaned up when the `FetchedGame` object is GCed.

Both classes provide a `.path` attribute which evaluates to a string giving the path of the `.z8` file.

In [8]:
class GeneratedGame:
    def __init__(
        self,
        seed : int,
        tw_make_args : List[str],
        tw_make_bin : str = TW_MAKE_BIN,
        game_dir : str = os.path.join(WORK_DIR, "games"),
    ):
        game_file = "{}-{}.z8".format(
            hashlib.sha256("\0".join(tw_make_args).encode("utf-8")).hexdigest(),
            seed
        )
        os.makedirs(game_dir, exist_ok=True)
        self.path : str = os.path.join(game_dir, game_file)
        if os.path.exists(self.path):
            self.have_info = os.path.exists(
                os.path.splitext(self.path)[0] + ".json"
            )
            return
        args = [tw_make_bin] + tw_make_args + \
            ["--output", self.path, "--seed", str(seed)]
        subprocess.run(args).check_returncode()
        self.have_info = True

class FetchedGame:
    def __init__(
        self,
        z8_url: str,
        json_url: Optional[str] = None
    ):
        self._tempdir = tempfile.TemporaryDirectory()
        self.path = os.path.join(self._tempdir.name, "game.z8")
        z8_request = urllib.request.urlopen(z8_url)
        with open(self.path, "wb") as f:
            f.write(z8_request.read())
        if json_url is not None:
            json_request = urllib.request.urlopen(json_url)
            with open(os.path.join(self._tempdir.name, "game.json"), "wb") as f:
                f.write(json_request.read())
            self.have_info = True
        else:
            self.have_info = False

Game = Union[GeneratedGame, FetchedGame]

`GameRunner` is a wrapper around a TextWorld gym environment which strips the parenthetical thoughts that we'll be prompting LLMs to include in their output, and also incorporates workarounds for a couple TextWorld bugs. A `GameRunner` can be constructed from an existing `GeneratedGame` or `FetchedGame` object, or from arguments to the constructors for the same. Its `reset` method starts a new game and returns a string containing the game's opening text. Its `step` method takes player input as a string, and returns a tuple `(obs, outcome)`. `obs` is a string containing the game's response to the input. `outcome` is one of "won", "lost", "quit" or "turnmax" if the game has ended, indicating respectively that the player won the game, lost, quit out, or hit the configured turn limit. An outcome of `False` indicates that the game is still in progress. If the game's .json metadata is not available, then it is not possible to determine game status and `outcome` will be `None`.

The `step` method will throw a `GameRunnerException` if the player input contains no command, multiple commands, or imbalanced parentheses. The rejection of multiple commands is a workaround for https://github.com/microsoft/TextWorld/issues/366. These exceptions are intended to be caught, and their messages are formatted as to be provided directly to the LLM agent as a developer prompt instructing the agent to correct its output. They also include an `old_message` attribute with a legacy wording which is less effective at getting the agent to correct its behavior, but can be used when reproducing the first of the two experiments in this notebook.

The `_clean_obs` method is a workaround for another, minor TextWorld bug, wherein the observation that the gym returns includes, without delimiters, text that the Z-machine intended to place into an overhead status bar. This is ugly and not useful so we strip it away.

In [9]:
class GameRunnerException(Exception):
    def __init__(self, message):
        super().__init__(message)
        self.old_message = message

class ImbalancedParensException(GameRunnerException):
    def __init__(self):
        super().__init__(
            message="I couldn't interpret your last message because it "
                "contained imbalanced parentheses. Please try again."
        )

class NoCommandException(GameRunnerException):
    def __init__(self):
        super().__init__(
            message="Your last message contained only a parenthetical, so there was "
                "no command to execute. Please try again, placing your thoughts "
                "inside parenthesis and the game command outside them."
        )
        self.old_message = \
            "Your last message contained nothing except a parenthetical,\n" \
            "so it couldn't be provided to the game. Try again. If you've \n" \
            "completely given up, just say QUIT.\n\n>"

class MultipleCommandException(GameRunnerException):
    def __init__(self):
        super().__init__(
            message="Your last message appeared to contain a series of multiple "
            "commands. The game doesn't support this. Please try again, issuing "
            "just one command at a time."
        )
        self.old_message = \
            "Your last message looks like you may have tried to issue\n" \
            "multiple commands at once. The game doesn't support this.\n" \
            "Try again, one command at a time."

GameRunnerOutcome = Literal[None, False, "won", "lost", "quit", "turnmax"]

class GameRunner:
    def _clean_obs(self, obs: str) -> str:
        return re.sub(r">[^>]*$", "> ", obs)

    def __init__(
            self,
            game: Optional[Game] = None,
            seed: Optional[int] = None,
            tw_make_args: Optional[List[str]] = None,
            tw_make_bin: str = TW_MAKE_BIN,
            z8_url: Optional[str] = None,
            json_url: Optional[str] = None,
            max_turns : int = 100
        ):
            if game is not None:
                self.game : Game = game
                if seed is not None:
                    raise ValueError("Cannot specify both game and seed")
                if z8_url is not None:
                    raise ValueError("Cannot specify both game and z8_url")
            elif seed is not None:
                if tw_make_args is None:
                    raise ValueError("Must specify tw_make_args if specifying seed")
                self.game = GeneratedGame(seed, tw_make_args, tw_make_bin)
                if z8_url is not None:
                    raise ValueError("Cannot specify both seed and z8_url")
            elif z8_url is not None:
                self.game = FetchedGame(z8_url, json_url)
            else:
                raise ValueError("Must specify either game, seed, or z8_url")

            if self.game.have_info:
                request_infos = textworld.core.EnvInfos(lost=True, won=True)
            else:
                request_infos = None

            self._env_id = textworld.gym.register_game(
                self.game.path,
                request_infos=request_infos,
                name=self.game.path,
                max_episode_steps=max_turns,
            )
            self._env = textworld.gym.make(self._env_id)

    def reset(self) -> str:
        obs, _ = self._env.reset()
        return self._clean_obs(obs)

    def step(self, action) -> Tuple[str, GameRunnerOutcome]:
        # Strip balanced, innermost parentheticals until none remain
        while True:
            new_action = re.sub(r"[(][^()]*[)]", "", action)
            if new_action == action:
                break
            action = new_action
        if "(" in action or ")" in action:
            raise ImbalancedParensException()
        action = action.lstrip().rstrip(". \r\n")
        if action == "":
            raise NoCommandException()
        if '.' in action or ',' in action or ';' in action or '\n' in action or \
                re.search(r"\b(THEN|AND)\b", action, re.IGNORECASE):
            raise MultipleCommandException()
        if re.match(r"(QUIT|RESTART)\b", action, re.IGNORECASE):
            self._env.close()
            return ("", "quit")
        obs, _, done, infos = self._env.step(action)
        obs = self._clean_obs(obs)

        if done:
            if infos.get('won', False):
                return (obs, 'won')
            elif infos.get('lost', False):
                return (obs, 'lost')
            else:
                return (obs, 'turnmax')
        elif self.game.have_info:
            return (obs, False)
        else:
            return (obs, None)

`AgentRunner` wraps the interaction with the LLM agent. It is constructed from an OpenAI client object and a `ProcessedModelSpec`, which is a dictionary type. Most of the entries in a `ProcessedModelSpec` are passed as keyword arguments to the OpenAI `chat.completions.create` API endpoint. It also takes a `developer_role` key which can specify a different role to substitute for "developer" when sending a developer prompt. This can be used to change the deveolper prompt into a user prompt in order to support models such as `o1-mini` which don't support developer prompts.

`AgentRunner` provides various `append` methods for adding onto the chat transcript, and a `run` method for invoking the OpenAI API to get the next completion. The completion is returned from `run` and also added onto the transcript. The `transcript` method returns a copy of the entire transcript so far.

In [10]:
class TranscriptItem(TypedDict):
    role: str
    content: str
Transcript = List[TranscriptItem]
TranscriptIterable = Iterable[TranscriptItem]

class BaseModelSpec(TypedDict):
    developer_role: NotRequired[str]
    frequency: NotRequired[float]
    logit_bias: NotRequired[Dict[str, int]]
    logprobs: NotRequired[bool]
    max_completion_tokens: NotRequired[int]
    max_tokens: NotRequired[int]
    presence_penalty: NotRequired[float]
    reasoning_effort: NotRequired[Literal['low', 'medium', 'high']]
    seed: NotRequired[int]
    service_tier: NotRequired[Literal['auto', 'default']]
    stop: NotRequired[Union[Optional[str], List[str]]]
    temperature: NotRequired[float]
    top_logprobs: NotRequired[int]
    top_p: NotRequired[float]

class ProcessedModelSpec(BaseModelSpec):
    model: str
    messages: NotRequired[Transcript]

class AgentRunner:
    def __init__(self, client: openai.OpenAI, modelspec: ProcessedModelSpec):
        self._client = client
        self._args = modelspec.copy()
        if 'developer_role' in self._args:
            self._developer_role = self._args['developer_role']
            del self._args['developer_role']
        else:
            self._developer_role = 'developer'
        self._args['messages'] = []

    def append(self, message: TranscriptItem):
        if message['role'] == 'developer':
            self.append_developer(message['content'])
        else:
            self._args['messages'].append(message)

    def extend(self, messages: TranscriptIterable):
        for message in messages:
            self.append(message)

    def append_developer(self, message: str):
        self._args['messages'].append({
            'role': self._developer_role,
            'content': message,
        })

    def append_user(self, message: str):
        self._args['messages'].append({
            'role': 'user',
            'content': message,
        })

    def append_assistant(self, message: str):
        self._args['messages'].append({
            'role': 'assistant',
            'content': message,
        })

    def run(self, render_errors=False) -> str:
        backoff = 15.
        backoff_ratio = 2.
        failures = 0

        while True:
            try:
                response = self._client.chat.completions.create(**cast(Any, self._args))
                break
            except (openai.RateLimitError, openai.APITimeoutError) as e:
                if render_errors:
                    print(e)
                fuzzed_backoff = backoff * random.uniform(0.75, 1.333)
                backoff *= backoff_ratio
                time.sleep(fuzzed_backoff)
            except openai.APIError as e:
                if render_errors:
                    print(e)
                failures += 1
                if failures >= 3:
                    raise e
                time.sleep(random.uniform(3, 7))
        content = response.choices[0].message.content
        self.append_assistant(content)
        return content

    def transcript(self) -> Transcript:
        return copy.deepcopy(self._args['messages'])

`AgentRunnerFactory` generates `AgentRunner`s from higher-level configuration structures and initializes them with an appropriate prompt. It is constructed from a dictionary of `ClientSpec`s, a dictionary of `ModelSpec`s, and a `PromptSpec`. Each `ClientSpec` entry maps an identifying string to a dictionary of keyword arguments provided to the `openai.OpenAI` constructor. These arguments typically need to include at least a `base_url` and an `api_key`. A `ModelSpec` contains mostly the same entries as the `ProcessedModelSpec` above, and also must include a `client` entry, which is a reference into the `ClientSpec` dictionary indicating which client should be used to interact with this model. A `PromptSpec` contains `instructions` and a list of `sample_games`. Each element of the `sample_games` list contains arguments for constructing a `GeneratedGame` or a `FetchedGame`, and a `solution` which is a list of inputs which solve that game. The resulting prompt will contain the instructions, then walkthroughs constructed from the given solutions, and end with a repetition of the instructions. The factory's `build` method takes a `model` argument (a reference into the dictionary of `ModelSpec`s) and returns an `AgentRunner`.

In [11]:
class ClientSpec(TypedDict):
    api_key: NotRequired[str]
    organization: NotRequired[str]
    project: NotRequired[str]
    base_url: NotRequired[str]
    websocket_base_url: NotRequired[str]
    timeout: NotRequired[float]
    max_retries: NotRequired[int]
    default_headers: NotRequired[Mapping[str, str]]
    default_query: NotRequired[Mapping[str, object]]

class ModelSpec(BaseModelSpec):
    client: str
    model: NotRequired[str]
    reasoner: NotRequired[bool]

class FetchedSampleGame(TypedDict):
    url: str
    solution: List[str]
    json_url: NotRequired[str]

class GeneratedSampleGame(TypedDict):
    seed: int
    tw_make_args: List[str]
    solution: List[str]

SampleGame = Union[FetchedSampleGame, GeneratedSampleGame]

class PromptSpec(TypedDict):
    instructions: str
    sample_games: List[SampleGame]

class AgentRunnerFactory:
    def __init__(
            self,
            client_specs: Dict[str, ClientSpec],
            model_specs: Dict[str, ModelSpec],
            prompt_spec: PromptSpec,
            tw_make_bin : str = TW_MAKE_BIN,
            game_dir : str = os.path.join(WORK_DIR, "games"),
        ):
        self._client_specs = client_specs.copy()
        self._model_specs = model_specs.copy()
        self._prompt_spec = prompt_spec.copy()
        self._tw_make_bin = tw_make_bin
        self._game_dir = game_dir
        self._nonreasoner_prompt : Optional[Transcript] = None
        self._reasoner_prompt : Optional[Transcript] = None

        self._samples : List[Transcript] = []
        for sample_spec in self._prompt_spec['sample_games']:
            sample : Transcript = []
            if 'url' in sample_spec:
                sample_spec = cast(FetchedSampleGame, sample_spec)
                game : Game = FetchedGame(
                    sample_spec['url'],
                    sample_spec.get('json_url')
                )
            elif 'seed' in sample_spec:
                sample_spec = cast(GeneratedSampleGame, sample_spec)
                game = GeneratedGame(
                    sample_spec['seed'],
                    sample_spec['tw_make_args'],
                    tw_make_bin=self._tw_make_bin,
                    game_dir=self._game_dir,
                )
            else:
                raise ValueError(
                    "Sample game must contain a url or a seed and tw_make_args"
                )
            game_runner = GameRunner(
                game=game,
                max_turns=len(sample_spec['solution']) + 1
            )
            sample.append({'role': 'user', 'content': game_runner.reset()})
            outcome = None
            for step in sample_spec['solution']:
                assert not outcome
                sample.append({'role': 'assistant', 'content': step})
                obs, outcome = game_runner.step(step)
                sample.append({'role': 'user', 'content': obs})
            assert outcome != False
            sample.append({'role': 'assistant', 'content': "QUIT"})
            self._samples.append(sample)

    def _get_nonreasoner_prompt(self) -> Transcript:
        if self._nonreasoner_prompt is not None:
            return self._nonreasoner_prompt
        nonreasoner_prompt = [{
            'role': 'developer',
            'content': self._prompt_spec['instructions']
        }]
        for sample in self._samples:
            nonreasoner_prompt.extend(sample)
            nonreasoner_prompt.append({
                'role': 'developer',
                'content': "Well done! Now play again. Your instructions are the "
                    "same as before:\n\n" + self._prompt_spec['instructions']
            })

        self._nonreasoner_prompt = nonreasoner_prompt
        return self._nonreasoner_prompt

    def _get_reasoner_prompt(self) -> Transcript:
        if self._reasoner_prompt is not None:
            return self._reasoner_prompt
        instructions = self._prompt_spec['instructions']
        if len(self._samples) == 1:
            instructions += "\n\nHere is a walkthrough of an example game:\n"
        elif len(self._samples) > 1:
            instructions += "\n\nHere are some walkthroughs of example games:\n"

        for sample in self._samples:
            instructions += "```\n"
            for message in sample:
                instructions += message['content']
                if message['role'] == 'assistant':
                    instructions += "\n"
            instructions += "```\n"

        if len(self._samples) > 0:
            instructions += "Now it's your turn!"

        self._reasoner_prompt = [{
            'role': 'developer',
            'content': instructions
        }]
        return self._reasoner_prompt

    def get_prompt(self, model_name: str) -> Transcript:
        model_spec = self._model_specs[model_name]
        if model_spec.get('reasoner', False):
            return self._get_reasoner_prompt()
        else:
            return self._get_nonreasoner_prompt()

    def build(self, model_name: str) -> AgentRunner:
        model_spec = self._model_specs[model_name]
        client_spec = self._client_specs[model_spec['client']]

        processed_spec : dict = cast(dict, model_spec.copy())
        if 'client' in processed_spec:
            del processed_spec['client']
        if 'reasoner' in processed_spec:
            del processed_spec['reasoner']
        if 'model' not in processed_spec:
            processed_spec['model'] = model_name

        client = openai.OpenAI(**client_spec)
        runner = AgentRunner(client, cast(ProcessedModelSpec, processed_spec))
        if model_spec.get('reasoner', False):
            runner.extend(self._get_reasoner_prompt())
        else:
            runner.extend(self._get_nonreasoner_prompt())
        return runner

The `ExperimentRunner` puts everything together. An `ExperimentRunner` is constructed from a `ClientSpec` dictionary, a `ModelSpec` dictionary, instructions, a list of sample games, a list of `tw-make` arguments, and a turn limit. It can then `run` an experiment, given a list of models to be tested and a list of game seeds to test them against. Each model will play each seed up to `max_attempts` (default 3) times until it wins. Results are stored in `$WORK_DIR/experiments/<model>/<spec-hash>/<seed>/<attempt>.json`. The spec-hash is computed from the model spec, the tw-make arguments, and the prompt spec (containing instructions and sample games), and all these hash inputs are also recorded in a file named `spec.json`. Unless `run`'s `force_rerun` argument is set to `True`, any (model, spec, seed, attempt) combination will only be run once, and subsequent `run` invocations will just return those stored results.

The entire game transcript is recorded as part of its result, but its outcome is summarized as one of:

* "won": The game was won.
* "lost": The game was lost, e.g. by improperly preparing an ingredient.
* "turnmax": The turn limit was reached before any other outcome.
* "quit": The agent quit out of the game befor otherwise finishing it.
* "silence": The agent output something uninterpretable for five turns in a row.
* "error": Either the game crashed, or OpenAI APIs calls errored out several times in a row.

In [12]:
Outcome = Literal['won', 'lost', 'turnmax', 'silence', 'quit', 'error']
class GameResult(TypedDict):
    model: str
    tw_make_args: List[str]
    seed: int
    error: Optional[str]
    outcome: Outcome
    messages: Transcript
    turns: int

class ExperimentRunner:
    def __init__(
            self,
            client_specs: Dict[str, ClientSpec],
            model_specs: Dict[str, ModelSpec],
            instructions: str,
            sample_games: List[SampleGame],
            tw_make_args: List[str],
            max_turns: int  = 100,
            old_error_wording: bool = False,
            max_silences: int = 5,
            tw_make_bin : str = TW_MAKE_BIN,
            experiment_dir : str = os.path.join(WORK_DIR, "experiments"),
            game_dir : str = os.path.join(WORK_DIR, "games"),
        ):
        self._client_specs = copy.deepcopy(client_specs)
        self._model_specs = copy.deepcopy(model_specs)
        self._prompt_spec : PromptSpec = {
            'instructions': instructions,
            'sample_games': sample_games,
        }
        self._tw_make_args = tw_make_args.copy()
        self._max_turns = max_turns
        self._old_error_wording = old_error_wording
        self._max_silences = max_silences
        self._tw_make_bin = tw_make_bin
        self._experiment_dir = experiment_dir
        self._game_dir = game_dir
        self._agent_runner_factory = AgentRunnerFactory(
            self._client_specs,
            self._model_specs,
            self._prompt_spec,
            tw_make_bin=self._tw_make_bin,
            game_dir=self._game_dir,
        )

    def experiment_spec(self, model: str) -> dict:
        canonicalized_model_spec = cast(dict, self._model_specs[model].copy())
        if 'model' not in canonicalized_model_spec:
            canonicalized_model_spec['model'] = model
        del canonicalized_model_spec['client']

        experiment_spec = {
            'model_spec': canonicalized_model_spec,
            'prompt_spec': self._prompt_spec,
            'tw_make_args': self._tw_make_args,
            'max_turns': self._max_turns,
            'old_error_wording': self._old_error_wording,
            'max_silences': self._max_silences,
        }

        hash_input = json.dumps(experiment_spec, sort_keys=True)
        hash = hashlib.sha256(hash_input.encode()).hexdigest()
        experiment_spec['hash'] = hash
        return experiment_spec

    def write_spec(self, model: str):
        experiment_spec = self.experiment_spec(model)
        model_dir = os.path.join(self._experiment_dir, model, experiment_spec['hash'])
        os.makedirs(model_dir, exist_ok=True)
        spec_file = os.path.join(model_dir, "spec.json")
        if not os.path.exists(spec_file):
            json.dump(experiment_spec, open(spec_file, 'w'), indent=4, sort_keys=True)

    def get_prompt(self, model: str):
        return self._agent_runner_factory.get_prompt(model)

    def _interact(self, agent: AgentRunner, game: GameRunner, render=False) -> dict:
        obs = game.reset()
        agent.append_user(obs)
        if render:
            print(obs)
        outcome = None
        silences = 0
        turns = 0
        while not outcome:
            try:
                action = agent.run(render_errors=render)
            except openai.APIError as e:
                return {
                    'error': str(e),
                    'outcome': 'error',
                    'messages': agent.transcript(),
                    'turns': turns,
                }
            if render:
                print(action)

            turns += 1

            try:
                obs, outcome = game.step(action)
            except GameRunnerException as e:
                silences += 1
                if silences >= self._max_silences:
                    return {
                        'error': None,
                        'outcome': 'silence',
                        'messages': agent.transcript(),
                        'turns': turns,
                    }
                if self._old_error_wording:
                    agent.append_developer(e.old_message)
                else:
                    agent.append_developer(str(e))
                continue
            agent.append_user(obs)
            if render:
                print(obs)
            silences = 0

        return {
            'error': None,
            'outcome': outcome,
            'messages': agent.transcript(),
            'turns': turns,
        }

    def run_single(
            self,
            model: str,
            seed: int,
            attempt: int = 0,
            render: bool = False,
            force_rerun: Optional[bool] = None,
            save: bool = True,
    ) -> GameResult:
        experiment_spec = self.experiment_spec(model)
        model_dir = os.path.join(self._experiment_dir, model, experiment_spec['hash'])
        seed_dir = os.path.join(model_dir, str(seed))
        attempt_path = os.path.join(seed_dir, str(attempt) + ".json")

        os.makedirs(seed_dir, exist_ok=True)
        if force_rerun != True and os.path.exists(attempt_path):
            return json.load(open(attempt_path, 'r'))
        if force_rerun == False:
            raise RuntimeError(
                "Failed to load model {} spec {} seed {} attempt {}".format(
                    model,
                    experiment_spec['hash'],
                    seed,
                    attempt
                )
            )

        agent_runner = self._agent_runner_factory.build(model)
        game = GeneratedGame(
            seed,
            self._tw_make_args,
            tw_make_bin=self._tw_make_bin,
            game_dir=self._game_dir,
        )
        game_runner = GameRunner(game=game, max_turns=self._max_turns)
        result = self._interact(agent_runner, game_runner, render=render)
        result['model'] = model
        result['tw_make_args'] = self._tw_make_args.copy()
        result['seed'] = seed
        if save:
            json.dump(result, open(attempt_path, 'w'), indent=4, sort_keys=True)
        return cast(GameResult, result)

    def run_to_success(
            self,
            model: str,
            seed: int,
            max_attempts: int = 3,
            max_errors: int = 3,
            render: bool = False,
            force_rerun: Optional[bool] = None,
            save: bool = True,
    ) -> List[GameResult]:
        nonerrored_attempts = 0
        errored_attempts = 0
        attempt = 0
        results = []

        while nonerrored_attempts < max_attempts and errored_attempts < max_errors:
            result = self.run_single(
                model,
                seed,
                attempt=attempt,
                render=render,
                force_rerun=force_rerun,
                save=save)
            results.append(result)
            if result['outcome'] == 'won':
                return results
            elif result['outcome'] == 'error':
                errored_attempts += 1
            nonerrored_attempts += 1
            attempt += 1
        return results

    def pregenerate_games(
            self,
            seeds: Union[int, Iterable[int]],
        ):
        if isinstance(seeds, int):
            seeds = [seeds]
        with multiprocessing.Pool() as pool:
            pool.starmap(
                _pregenerate_game_worker,
                 [
                     (seed, self._tw_make_args, self._tw_make_bin, self._game_dir)
                    for seed in seeds
                ]
            )

    def run(
            self,
            models: Union[str, Iterable[str]],
            seeds: Union[int, Iterable[int]],
            max_attempts: int = 3,
            max_errors: int = 3,
            processes: int = 1,
            render: bool = False,
            force_rerun: Optional[bool] = None,
            save: bool = True,
        ) -> List[List[GameResult]]:

        if isinstance(models, str):
            models = [models]
        if isinstance(seeds, int):
            seeds = [seeds]

        for model in models:
            self.write_spec(model)
        self.pregenerate_games(seeds)
        with multiprocessing.Pool(processes=processes) as pool:
            return pool.starmap(
                _run_worker,
                [
                    (self, model, seed, max_attempts, max_errors, force_rerun, save)
                    for model in models
                    for seed in seeds
                ]
            )

    def download(self):
        try:
            from google.colab import files
        except ImportError as e:
            raise RuntimeError(
                "Download failed: this notebook is not running in Google Colab."
            ) from e

        os.makedirs(self._experiment_dir, exist_ok=True)
        with tempfile.TemporaryDirectory() as temp_dir:
            tar_filename = os.path.join(temp_dir, f"experiment.tar.gz")
            with tarfile.open(tar_filename, "w:gz") as tar:
                tar.add(
                    self._experiment_dir,
                    arcname=os.path.basename(self._experiment_dir)
                )

            files.download(tar_filename)

def _pregenerate_game_worker(seed, tw_make_args, tw_make_bin, game_dir):
    GeneratedGame(
        seed,
        tw_make_args,
        tw_make_bin=tw_make_bin,
        game_dir=game_dir,
    )

def _run_worker(
        runner, model, seed, max_attempts, max_errors, force_rerun, save
    ) -> List[GameResult]:
    return runner.run_to_success(
        model, seed, max_attempts, max_errors, force_rerun=force_rerun, save=save
    )



## Analysis code

The following code generates performance summaries for each model, determining their win rate by their n'th attempt.

The `credible_intervals` method generate Bayesian credible intervals on these wins rates, treating each seed as a uniquely-biased coin drawn from a population with an unknown distribution of biases. For the first attempt, the bounds of the credible interval are simply quantiles of a beta distribution with parameters $\alpha = \textrm{wins} + 0.5$ and $\beta = \textrm{losses} + 0.5$, but since we're retrying specifically those seeds which were failed on earlier attempts, the later distributions have no clean analytic form, so numerical methods are required; we use a Monte Carlo sim.

The `test_improvement` method tests whether one experiment produced a statistically significant improvement over another, using a single-tailed stratified Fisher exact test, which is an exact alternative to the Cochran–Mantel–Haenszel test. This exactitude comes at the cost of computational complexity: for $M$ models with $N$ seeds tested, it is $O(M\cdot N^M)$. Fortunately, for $M=4$ and $N=100$ this is still tractable and should execute in a few seconds.

In [13]:
import numpy as np, scipy.stats

In [14]:
OutcomeCounts = Dict[Outcome, int]
OutcomeList = List[OutcomeCounts]
OutcomeTable = Dict[str, OutcomeList]

class Analyzer:
    def __init__(self, all_results: List[List[GameResult]]):
        self._all_results = all_results
        self._filtered_results = []
        for result_list in self._all_results:
            filtered = [ result for result in result_list if result['outcome'] != 'error']
            self._filtered_results.append(filtered)
        self._results_by_model : Dict[str, List[List[GameResult]]] = dict()

        self._max_attempts = 0
        for results_list in self._filtered_results:
            self._max_attempts = max(self._max_attempts, len(results_list))

        for results_list in self._filtered_results:
            model = results_list[0]['model']
            if model not in self._results_by_model:
                self._results_by_model[model] = list()
            self._results_by_model[model].append(results_list)

        self._outcomes : OutcomeTable = dict()

        for model, all_model_results in self._results_by_model.items():
            model_outcomes : OutcomeList = [dict() for _ in range(self._max_attempts)]
            for attempt in range(self._max_attempts):
                for results in all_model_results:
                    if attempt < len(results):
                        outcome = results[attempt]['outcome']
                        if outcome not in model_outcomes[attempt]:
                            model_outcomes[attempt][outcome] = 0
                        model_outcomes[attempt][outcome] += 1

            self._outcomes[model] = model_outcomes

        self._cumulative_outcomes : OutcomeTable = dict()
        for model, outcomes in self._outcomes.items():
            incremental_outcome = self._outcomes[model][0].copy()
            self._cumulative_outcomes[model] = [incremental_outcome.copy()]
            for attempt in range(1, self._max_attempts):
                for outcome, count in self._outcomes[model][attempt].items():
                    if outcome not in incremental_outcome:
                        incremental_outcome[outcome] = 0
                    incremental_outcome[outcome] += count
                self._cumulative_outcomes[model].append(incremental_outcome.copy())


    def outcomes(self) -> OutcomeTable:
        return copy.deepcopy(self._outcomes)

    def cumulative_outcomes(self) -> OutcomeTable:
        return copy.deepcopy(self._cumulative_outcomes)

    def _model_credible_intervals(
            self,
            model : str,
            prior_a : float = 0.5,
            prior_b : float = 0.5,
            conf_level : float = 0.95,
            nsamples: int = 1000000
        ) -> List[Tuple[float, float]]:

        intervals = []
        cumulative = np.zeros(nsamples)
        lower_quantile = (1 - conf_level) / 2.0
        upper_quantile = (1 + conf_level) / 2.0
        remaining = len(self._results_by_model[model])
        wins_by_attempt = [
            self._outcomes[model][n].get('won',0)
            for n in range(self._max_attempts)
        ]

        for wins in wins_by_attempt:
            a = wins + prior_a
            b = (remaining - wins) + prior_b
            p_samples = scipy.stats.beta.rvs(a, b, size=nsamples)
            cumulative += (1 - cumulative) * p_samples
            ci = np.quantile(cumulative, [lower_quantile, upper_quantile])
            intervals.append((ci[0], ci[1]))
            remaining -= wins

        return intervals

    def credible_intervals(
            self,
            prior_a : float = 0.5,
            prior_b : float = 0.5,
            conf_level : float = 0.95,
            nsamples: int = 1000000
        ) -> Dict[str, List[Tuple[float, float]]]:
        intervals = dict()
        for model in self._outcomes.keys():
            intervals[model] = self._model_credible_intervals(
                model, prior_a, prior_b, conf_level, nsamples
            )
        return intervals

    def _stratified_fisher_exact(
            self,
            control : np.ndarray,
            treatment: np.ndarray
        ) -> float:

        # Validate input shapes
        M = control.shape[0]
        if control.shape != (M, 2) or treatment.shape != (M, 2):
            raise ValueError("control and treatment must be of shape (M, 2)")

        # Compute observed test statistic (sum of a_i)
        S_observed = np.sum(treatment[:, 0])

        # Lists to store bounds and probabilities for each stratum
        lb_list = []
        ub_list = []
        probs_list = []

        # Process each stratum
        for i in range(M):
            # Compute row and column totals
            n1_i = treatment[i, 0] + treatment[i, 1]  # Treatment total
            n2_i = control[i, 0] + control[i, 1]      # Control total
            m1_i = treatment[i, 0] + control[i, 0]    # Responsive total
            N_i = n1_i + n2_i                          # Grand total

            # Determine range for a_i
            lb_i = max(0, n1_i + m1_i - N_i)
            ub_i = min(n1_i, m1_i)

            # Compute possible values of a_i and their probabilities
            possible_a_i = np.arange(lb_i, ub_i + 1)
            probs_i = scipy.stats.hypergeom.pmf(possible_a_i, N_i, m1_i, n1_i)

            lb_list.append(lb_i)
            ub_list.append(ub_i)
            probs_list.append(probs_i)

        # Compute p-value by enumerating all possible combinations
        p_value = 0.0
        for combination in itertools.product(
                *[range(lb, ub + 1) for lb, ub in zip(lb_list, ub_list)]
            ):
            # Compute sum of a_i for this combination
            S = sum(combination)
            # Compute probability as product across strata
            P = np.prod([probs_list[i][combination[i] - lb_list[i]] for i in range(M)])
            # Add to p-value if S is as extreme or more extreme
            if S >= S_observed:
                p_value += P

        return p_value

    def test_improvement(self, newer):
        common_models = frozenset(self._results_by_model.keys()) & \
            frozenset(newer._results_by_model.keys())
        control = []
        treatment = []

        for model in common_models:
            control_total = sum(self._outcomes[model][0].values())
            treatment_total = sum(newer._outcomes[model][0].values())
            control_wins = self._outcomes[model][0].get('won', 0)
            treatment_wins = newer._outcomes[model][0].get('won', 0)
            control_losses = control_total - control_wins
            treatment_losses = treatment_total - treatment_wins
            control.append((control_wins, control_losses))
            treatment.append((treatment_wins, treatment_losses))
        return self._stratified_fisher_exact(np.asarray(control), np.asarray(treatment))


## First Experiment

The first of the two experiments used a naively-written prompt, crafted based on usual best practices for prompt engineering but without the benefit of seeing how it would perform, and a walkthrough derived from the sample game on the TextWorld website. Tested models included gpt-4o (version 2024-08-06), gpt-4o-mini (version 2024-07-18), Llama3.1-405B-Instruct-FP8, and Llama3.3-70B-Instruct-FP8.

In [15]:
CLIENTS : Dict[str, ClientSpec] = {
    'openai': {
        'base_url': "https://api.openai.com/v1",
        'api_key': get_api_key("openai_api_key"),
    },
    'lambdalabs': {
        'base_url': "https://api.lambdalabs.com/v1",
        'api_key': get_api_key("lambdalabs_api_key"),
    },
}

MODELS : Dict[str, ModelSpec] = {
    'gpt-4o': {'client': 'openai'},
    'gpt-4o-mini': {'client': 'openai'},
    'llama3.1-405b-instruct-fp8': {
        'client': 'lambdalabs'
    },
    'llama3.3-70b-instruct-fp8': {
        'client': 'lambdalabs',
    }
}

/tmp/ipykernel_3844/1270726539.py:12: UserWarning: Failed to retrieve API key for openai_api_key. Set it in Colab secrets or via the OPENAI_API_KEY environment variable.
  warnings.warn(
/tmp/ipykernel_3844/1270726539.py:12: UserWarning: Failed to retrieve API key for lambdalabs_api_key. Set it in Colab secrets or via the LAMBDALABS_API_KEY environment variable.
  warnings.warn(


In [16]:
TW_MAKE_ARGS : List[str] = [
    "tw-cooking", "--recipe","3", "--take", "2", "--go", "12", "--open", "--cook",
    "--cut", "--drop",
]

In [17]:
MAX_TURNS : int = 100

In [18]:
INSTRUCTIONS : str = \
"""You are going to play a text adventure game. All of the user's input
represents text printed by the game. Your output will be interpreted
as commands issued to the game. However, the game will ignore
anything you put in parentheses. Use parentheticals to record your
thoughts as you think step-by-step through these instructions before
you decide what move to make. Every time you enter a new room, begin
your next thought by listing what exits you see.

The game you will be playing is randomly generated, but all games
follow the same simple template. Your goal is to prepare and eat a
meal. To do this, you will perform the following, three-phase
procedure.

# Phase I: Find the kitchen

Explore the map until you locate the kitchen. Use compass commands
`N`, `S`, `E`, and `W` to move around. There may be doors in your
way. Use the command `OPEN <adjective> DOOR` to open them,
substituting whatever adjective you see in the room description.

When you explore, pay careful attention to any occurrence of "north",
"south", "east", or "west" in a room description. These words always
indicate room exits. Explore all exits until you've located
everything you need.

During this phase, make a note of any items you come across, but do
not pick any of them up.

# Phase II: Search for ingredients

Once you find the kitchen, use the command `READ COOKBOOK`. The
cookbook will contain a list of ingredients, and then a list of
directions for preparation. Right now, just pay attention to the
list of ingredients. Don't worry about preparation until Phase III.

Next, use the command `I` to list your inventory. You may already be
carrying some of the ingredients you need. If you are carrying
anything that you *don't* need, use the `DROP` command to drop it.
You may find some items in your inventory that you didn't pick up,
which were there at the start of the game. This is normal.

Third, use the command `OPEN FRIDGE`. If the fridge contains any
ingredients you need, use the `GET` command to pick them up.

Now, figure out what ingredients you are still missing, and then
continue exploring the map until you find them.

Pick up only the ingredients that the recipe calls for. Pay
attention to their entire description. For example, if the recipe
calls for a red pepper, don't pick up a yellow pepper unles the
recipe needs that too.

If you try to pick something up and the game tells you, "You're
carrying too many things already", you've messed up: either you're
carrying something you don't need, or you don't need the thing
you're trying to pick up.  Check your inventory again (use `I`),
compare your inventory against the recipe, and `DROP` any
unnecessary items.

If you get lost or stuck in a loop while exploring in Phase II,
remember the principles from Phase I: look for "north", "south",
"east", and "west" in room descriptions to make sure you haven't
overlooked any exits. If you notice a closed door, that's certainly
a way you haven't explored yet.

# Phase III: Prepare your meal

Once you have all your ingredients, return to the kitchen. Now
`READ COOKBOOK` again, and then check your inventory again.
Double-check that your inventory contains all the required
ingredients and nothing else. If you made a mistake, return to
phase II and correct it.

Now you are ready to follow the directions in the cookbook. Each
step except the last one in the list of directions is either a
cutting step, or a cooking step. A cutting step calls for you either
to *slice*, *dice*, or *chop* an ingredient. A cooking step calls
for you either to *roast*, *fry*, or *grill* an ingredient.

Determine if any steps involve cutting. If and only if you need to
cut anything, use `GET KNIFE` to pick up the knife. You might have
to drop something first to make room for the knife in your inventory.

Now, follow the directions in order and follow them exactly. Use the
following commands:

* If the cookbook says to *slice* an ingredient, use `SLICE <ingredient> WITH KNIFE`.
* If the cookbook says to *dice* an ingredient, use `DICE <ingredient> WITH KNIFE`.
* If the cookbook says to *chop* an ingredient, use `CHOP <ingredient> WITH KNIFE`.
* If the cookbook says to *roast* an ingredient, use `COOK <ingredient> WITH OVEN`.
* If the cookbook says to *fry* an ingredient, use `COOK <ingredient> WITH STOVE`.
* If the cookbook says to *grill* an ingredient, use `COOK <ingredient> WITH BBQ`.

Be careful to use the right verb for cutting and the right appliance
for cooking! If you slice something that you should have diced or
chopped, or if you roast something that you should have fried or
grilled, you'll lose the game. Also be sure not to cook any
ingredient more than once, or you'll burn it.

Everything you need for meal preparation is in the kitchen, except
for the BBQ. The BBQ is in the backyard. So, if you have to grill
something, you will have to go to the backyard first and then return
to the kitchen afterward. After you get back to the kitchen, you
should `READ COOKBOOK` again so you can remember where you left off.

The final step in the list of directions will be "prepare meal", so
once you have done everything else, use `PREPARE MEAL` to put the
ingredients together; make sure you're still holding them all. You
can drop the knife now if you need to. Finally, use the command
`EAT MEAL` to win the game, and then `QUIT`."""

In [19]:
SAMPLE_GAMES : List[SampleGame] = [
    {
        'url': (
            "https://textworldparchmentserver.z13.web.core.windows.net/games/game1.z8"
        ),
        'solution': [
            "(I see exits to the south, east, and west. I'm in Phase I, searching for "
                "the kitchen. I've started the game in the backyard, where the BBQ is. "
                "That isn't important right now, but I'll remember it for later in "
                "case I need to grill something. Let's try the east exit first.) E",
            "(I see an exit to the west. Looks like this is a dead end with nothing "
                "useful, so I'll go back west) W",
            "(I see exits to the south, east, and west.  I'll search westward next, "
                "but first I need to open the door in my way) OPEN WOODEN DOOR",
            "(Now that the door is open, the path west is clear) W",
            "(I see an exit to the east. Another dead end. The toolbox and workbench "
                "aren't useful.) E",
            "(I see exits to the south, east, and west.  I've checked east and west, "
                "so only south remains. There's another door there I need to open.) "
                "OPEN SCREEN DOOR",
            "(Now my way south is clear.) S",
            "(I see exits to the north, south, east, and west. I found the kitchen, "
                "so I'm in Phase II now. The first thing to do in Phase II is read "
                "the cookbook.) READ COOKBOOK",
            "(Okay, I'm looking for a black papper, a chicken leg, and milk. Do I "
                "already have any of that?) I",
            "(No, there's nothing in my inventory. So there's nothing I need to drop, "
                "but three ingredients to search for. The fridge is the first place "
                "to search.) OPEN FRIDGE",
            "(That raw chicken leg is one of my three ingredients, so I should pick "
                "it up. The wing isn't an ingredient, so I'll ignore that.) "
                "GET RAW CHICKEN LEG",
            "(That's one ingredient down, two I'm still searching for. I see there "
                "are exits from here in all four directions, but I've already explored "
                "everything to the north. Let's try west.)  OPEN FROSTED-GLASS DOOR",
            "(Now my path west is clear.) W",
            "(I see an exit to the east. I don't need flour, so there's nothing useful "
                "here, and it's a dead end.) E",
            "(I see exits to the north, south, east, and west. The east and south "
                "exits are still unexplored. Let's go east.) E",
            "(I see exits to the south and west. There's nothing useful here, but "
                "there's an unexplored exit to the south.) S",
            "(I see exits to the north and west. Again nothing useful here, but "
                "another unexplored exit to the west.) W",
            "(I see exits to the north, east, and west. In my last three turns I went "
                "east, south, and west from the kitchen, so that north exit probably "
                "takes me back to the kitchen. West seems like the most promising "
                "direction for further exploration, but I need to open the door "
                "first.) OPEN FRONT DOOR",
            "(Now my way west is clear.) W",
            "(I see exits to the east and west. There's nothing useful here, but "
                "there's an explored exit to the west.) W",
            "(I see exits to the north and east. Again nothing useful here, but "
                "there's a door to the north.) OPEN SLIDING DOOR",
            "(Now my way north is clear.) N",
            "(I see an exit to the south. Jackpot! This room has both of the "
                "ingredients I'm still searching for. First, the black pepper.) "
                "GET BLACK PEPPER",
            "(I also need the milk.) GET MILK",
            "(I don't need the red hot pepper or the salt. I have all my ingredients "
                "now, which means I'm in Phase III and should return to the kitchen "
                "by going back the way I came.) S",
            "(I see exits to the north and east. The house is back east.) E",
            "(I see exits to the east and west. I'll go east again to re-enter the "
                "house.) E",
            "(I see exits to the north, east, and west. Okay, now I'm back in this "
                "corridor. I speculated earlier that north would be a shortcut back "
                "to the kitchen.) N",
            "(I see exits to the north, south, east, and west. I'm back in the "
                "kitchen, so it's time to re-read the cookbook.) READ COOKBOOK",
            "(Now I'll check my inventory to double-check that I have all the "
                "ingredients.) I",
            "(The ingredients I need are a black pepper, chicken leg, and milk. That "
                "exactly matches my inventory, so that means I did everything right "
                "so far and can proceed with the directions. Sice none of the "
                "directions involve cutting, I can ignore the knife. The first "
                "direction says to roast the chicken leg. When I need to roast "
                "something, I use the oven. I mustn't use the stove or the BBQ; "
                "I'd lose the game if I did that.)  COOK RAW CHICKEN LEG WITH OVEN",
            "(Okay, checking my inventory one more time: I should still be holding "
                "all the now-cooked ingredients.) I",
            "(Right. I'm still holding the now-roasted chicken leg, as well as milk "
                "and the black pepper which didn't need any preparation. Now, the "
                "final step in the directions is to prepare the meal, and I should be "
                "ready for that.) PREPARE MEAL",
            "(Finally, I can eat the meal. This should win the game.) EAT MEAL",
        ],
    }
]

In [20]:
FIRST_EXPERIMENT = ExperimentRunner(
    CLIENTS, MODELS, INSTRUCTIONS, SAMPLE_GAMES, TW_MAKE_ARGS, MAX_TURNS,
    old_error_wording=True, max_silences=3
)

In [21]:
FIRST_RESULTS = FIRST_EXPERIMENT.run(
    ['gpt-4o', 'gpt-4o-mini', 'llama3.3-70b-instruct-fp8', 'llama3.1-405b-instruct-fp8'],
    range(1,101),
    processes=12,
)

In [22]:
FIRST_ANALYSIS=Analyzer(FIRST_RESULTS)

In [23]:
FIRST_ANALYSIS.cumulative_outcomes()

{'gpt-4o': [{'won': 89, 'lost': 4, 'quit': 7},
  {'won': 98, 'lost': 4, 'quit': 9},
  {'won': 100, 'lost': 4, 'quit': 9}],
 'gpt-4o-mini': [{'won': 23,
   'lost': 13,
   'turnmax': 46,
   'silence': 10,
   'quit': 8},
  {'won': 30, 'lost': 20, 'turnmax': 91, 'silence': 16, 'quit': 20},
  {'won': 34, 'lost': 34, 'turnmax': 132, 'silence': 20, 'quit': 26}],
 'llama3.3-70b-instruct-fp8': [{'quit': 19,
   'won': 57,
   'turnmax': 18,
   'lost': 5,
   'silence': 1},
  {'quit': 24, 'won': 80, 'turnmax': 28, 'lost': 9, 'silence': 2},
  {'quit': 30, 'won': 85, 'turnmax': 35, 'lost': 10, 'silence': 2}],
 'llama3.1-405b-instruct-fp8': [{'won': 89,
   'quit': 8,
   'turnmax': 2,
   'silence': 1},
  {'won': 100, 'quit': 8, 'turnmax': 2, 'silence': 1},
  {'won': 100, 'quit': 8, 'turnmax': 2, 'silence': 1}]}

In [24]:
FIRST_ANALYSIS.credible_intervals()

{'gpt-4o': [(np.float64(0.8176599439045491), np.float64(0.9401947149438062)),
  (np.float64(0.9397817055253157), np.float64(0.9960141965954791)),
  (np.float64(0.9796514577389814), np.float64(0.9999963387696768))],
 'gpt-4o-mini': [(np.float64(0.15602731257411376),
   np.float64(0.319449950676573)),
  (np.float64(0.22086876029921393), np.float64(0.3993258761470935)),
  (np.float64(0.2610240006936727), np.float64(0.44552987489473))],
 'llama3.3-70b-instruct-fp8': [(np.float64(0.4720833257968957),
   np.float64(0.6639479025556048)),
  (np.float64(0.7165785737975244), np.float64(0.8708052193954307)),
  (np.float64(0.7772788956122636), np.float64(0.9133834308307244))],
 'llama3.1-405b-instruct-fp8': [(np.float64(0.8176389827271875),
   np.float64(0.940064104783628)),
  (np.float64(0.9763556037370433), np.float64(0.999995352549931)),
  (np.float64(0.9848338366767211), np.float64(0.999999740603284))]}

## Second experiment

For the second experiment, the instructions were slightly revised to place more emphasis on systematic exploration. The walkthrough from the TextWorld website's example game was replaced with one based on the same difficulty settings used for the test, using a seed selected for having a particularly difficult map in order to better illustrate how to navigate. Furthermore, the error messages presented to the agent when it provides uninterpretable input were revised, and the threshold for giving up after consecutive uninterpretable responses was raised from 3 to 5.

In [25]:
NEW_INSTRUCTIONS = \
"""You are going to play a text adventure game. All of the user's input represents
text printed by the game. On each turn, consider the information that was just
presented to you, plan your next move, and then make it. Place your thoughts
inside parentheses, and then outside those parentheses, issue a command to the
game. The game will interpret all of, and only, what is not parenthesized as a
command.

Maintain a calm and coherent tone at all times. Although the game's prose is often
awkward and disjointed, you should not imitate that style.

The game you will be playing is randomly generated, but all games follow the
same simple template. Your goal is to prepare and eat a meal. To do this, you
will perform a three-phase procedure. In the first phase, you will search for
the kitchen. In the second phase, you will read the cookbook to determine what
ingredients you need and then gather those ingredients. In the third phase,
you will return to the kitchen with the ingredients and then prepare and eat
the meal.

# Phase I: Find the kitchen

Explore the map until you locate the kitchen. Use compass commands `N`, `S`,
`E`, and `W` to move around. There may be doors in your way. Use the command
`OPEN <adjective> DOOR` to open them, substituting whatever adjective you see in
the room description.

When you explore, pay careful attention to any occurrence of "north", "south",
"east", or "west" in a room description. These words always indicate room exits.
Structure your exploration systematically and "depth-first", preferring first to
explore new avenues but backtracking once you've exhausted any particular
branch.

During this phase, make a note of any items you come across, but do not pick any
of them up.

# Phase II: Search for ingredients

Once you find the kitchen, use the command `READ COOKBOOK`. The cookbook will
contain a list of ingredients, and then a list of directions for preparation.
Right now, just pay attention to the list of ingredients. Don't worry about
preparation until Phase III.

Next, use the command `I` to list your inventory. You may already be carrying
some of the ingredients you need. If you are carrying anything that you *don't*
need, use the `DROP` command to drop it. You may find some items in your
inventory that you didn't pick up, which were there at the start of the game.
This is normal.

Third, use the command `OPEN FRIDGE`. If the fridge contains any ingredients you
need, use the `GET` command to pick them up.

Now, figure out what ingredients you are still missing, and then continue
exploring the map until you find them. Resume your exploration in the same
systematic fashion that you began it in the earlier phase.

Pick up only the ingredients that the recipe calls for. Pay attention to their
entire description. For example, if the recipe calls for a red pepper, don't
pick up a yellow pepper unles the recipe needs that too.

If you try to pick something up and the game tells you, "You're carrying too
many things already", you've messed up: either you're carrying something you
don't need, or you don't need the thing you're trying to pick up.  Check your
inventory again (use `I`), compare your inventory against the recipe, and `DROP`
any unnecessary items.

If you get stuck searching for an ingredient, don't give up: it's always
somewhere accessible and there's probably somewhere you forgot to explore.
Instead of wandering in circles, try clearing your mind and beginning a fresh,
systematic, depth-first traversal of the whole map.


# Phase III: Prepare your meal

Once you have all your ingredients, return to the kitchen. Now `READ COOKBOOK`
again, and then check your inventory again. Double-check that your inventory
contains all the required ingredients and nothing else. If you made a mistake,
return to phase II and correct it.

Now you are ready to follow the directions in the cookbook. Each step except the
last one in the list of directions is either a cutting step, or a cooking step.
A cutting step calls for you either to *slice*, *dice*, or *chop* an ingredient.
A cooking step calls for you either to *roast*, *fry*, or *grill* an ingredient.

Determine if any steps involve cutting. If and only if you need to cut anything,
use `GET KNIFE` to pick up the knife. You might have to drop something first to
make room for the knife in your inventory.

Now, follow the directions in order and follow them exactly. Use the following
commands:

* If the cookbook says to *slice* an ingredient, use `SLICE <ingredient> WITH KNIFE`.
* If the cookbook says to *dice* an ingredient, use `DICE <ingredient> WITH KNIFE`.
* If the cookbook says to *chop* an ingredient, use `CHOP <ingredient> WITH KNIFE`.
* If the cookbook says to *roast* an ingredient, use `COOK <ingredient> WITH OVEN`.
* If the cookbook says to *fry* an ingredient, use `COOK <ingredient> WITH STOVE`.
* If the cookbook says to *grill* an ingredient, use `COOK <ingredient> WITH BBQ`.

Be careful to use the right verb for cutting and the right appliance for
cooking! If you slice something that you should have diced or chopped, or if you
roast something that you should have fried or grilled, you'll lose the game.
Also be sure not to cook any ingredient more than once, or you'll burn it.

Everything you need for meal preparation is in the kitchen, except for the BBQ.
The BBQ is in the backyard. So, if you have to grill something, you will have to
go to the backyard first and then return to the kitchen afterward. After you get
back to the kitchen, you should `READ COOKBOOK` again so you can remember where
you left off.

The final step in the list of directions will be "prepare meal", so once you
have done everything else, use `PREPARE MEAL` to put the ingredients together;
make sure you're still holding them all. You can drop the knife now if you need
to. Finally, use the command `EAT MEAL` to win the game, and then `QUIT`."""

In [26]:
NEW_SAMPLE_GAMES = [{
    'seed': 65531,
    'tw_make_args': TW_MAKE_ARGS,
    'solution': [
        "(I've started in the bathroom. The only exit is to the south. I don't see "
            "anything useful here, so I'll just take that exit.) S",
        "(I see exits to the north, east, and west. I just came from the north, but "
            "east and west would be equally good ways to continue exploring. Let's "
            "try west first.) W",
        "(I see exits to the south, east, and west, with the east one being the way "
            "I just came and the south and west being unexplored. But since I've found "
            "the kitchen, I'm now in phase II and there are some things to do here "
            "before I continue exploring. The first step of phase II is to read the "
            "cookbook.) READ COOKBOOK",
        "(I'm looking for a block of cheese, a red bell pepper, and a red tuna. Now "
            "let's see what's already in my inventory.) I",
        "(My inventory contains a raw red tuna, a raw red potato, and a raw white "
            "tuna. The red tuna is one of the ingredients I need, but I don't need "
            "the other two items, so I'll drop those.) DROP RAW RED POTATO",
        "(I also need to drop the raw white tuna.) DROP RAW WHITE TUNA",
        "(Now that my inventory is squared away, I should search the fridge.) "
            "OPEN FRIDGE",
        "(Opening the fridge didn't report any contents, so it's probably empty, but "
            "I'll double-check.) LOOK IN FRIDGE",
        "(Yes, it's empty. Now it's time to continue searching for ingredients. I "
            "left off with unexplored exits to the west and south. I got here from the "
            "east, and there's also more back there to explore, but I'll continue "
            "forward first prior to backtracking. Let's go west.) W",
        "(Now I see exits to the north, east, and south, with the north and south "
            "being unexplored. There are no ingredients in this room, so I should "
            "continue exploring. Let's go north. Since there's a door in that "
            "direction, I first need to open it.) OPEN FRONT DOOR",
        "(Now my way north is clear.) N",
        "(I see exits to the north and south. There are no ingredients here. I came "
            "from the south, so north is the way to continue exploration.) N",
        "(I see exits to the south and east. Same situation as last turn: no "
            "ingredients, two exits, and one of those is back the way I came so "
            "I'll take the unexplored one to the east. That one has a door which I "
            "need to open.) OPEN COMMERCIAL GLASS DOOR",
        "(Now my way east is clear.) E",
        "(I've reached a dead end, with only one exit back to the west. However, that "
            "block of cheese is one of the ingredients I'm searching for, so I'll pick "
            "it up.) GET BLOCK OF CHEESE",
        "(Now I still need to locate the red bell pepper. Since I'm at a dead end, "
            "I need to begin backtracking.) W",
        "(Only two possible exits here, both explored, so I continue backtracking by "
            "taking the one to the south.) S",
        "(My situation is similar to last turn: I'm backtracking until I find "
            "somewhere new to explore. There's nowhere new here, so I'll continue "
            "backtracking to the south.) S",
        "(Now I'm back in the living room, which has exits to the north, south, and "
            "east. I just came from the north and have fully explored that branch now. "
            "Earlier I got here from the kitchen, which is to the east. But I haven't "
            "explored south from here yet, so I'll do that next.) S",
        "(This is a dead end, with the only exit back to the north. The red bell "
            "pepper isn't here, so there's nothing to do but go back.) N",
        "(I'm back in the living room again. I've finished exploring everything to "
            "the north and south, so now I need to backtrack east.) E",
        "(I'm back in the kitchen, but since I'm still missing the red bell pepper "
            "I need to keep exploring. There are exits to the west, south, and east. "
            "I just came from the west. East takes me back the way I got here "
            "originally during phase I. South is unexplored, so I should go that way "
            "after I've opened the door.) OPEN FROSTED-GLASS DOOR",
        "(Now my way south is clear.) S",
        "(Another dead end, and no red bell pepper. Nothing to do but go back north.) "
            "N",
        "(I've now thoroughly explored to the west and the south, so my only remaining "
            "option is to backtrack again to the east.) E",
        "(I'm now back in the corridor, where I haven't been since phase I. There are "
            "exits north, east, and west. I just came from the west, and at the very "
            "beginning of the game I got here from the north. The closed door to the "
            "east reminds me that I haven't checked there yet, so I should open it "
            "and then head that way.) OPEN SCREEN DOOR",
        "(Now my way east is clear.) E",
        "(I see exits to the west, south, and north. I also notice the BBQ here, which "
            "I don't need yet but I'll remember it's here. I just came from the west, "
            "and can explore to the north or south. Let's try north.) N",
        "(This is a dead end with the only exit back south, but I found the red bell "
            "pepper that I've been looking for, so I'll pick it up.) GET RED BELL "
            "PEPPER",
        "(That was my last ingredient, so now I'm in phase III. I can ignore the red "
            "onion and the green hot pepper and start heading back to the kitchen.) S",
        "(I'm back in the backyard. There are exits to the north, west and south. I "
            "just came from the north. The way back to the kitchen is west. I haven't "
            "explored south yet, but since I already have my ingredients and I also "
            "know where the BBQ is, there's no need for further exploration and I "
            "can continue back to the kitchen.) W",
        "(I'm back in the corridor again, with exits to the west, north, and east. "
            "West should take me to the kitchen.) W",
        "(I've reached the kitchen. Now for a sanity-check, I'm going to re-read "
            "the cookbook and then check my inventory.) READ COOKBOOK",
        "(The cookbook reminds me that I should be carrying a block of cheese, a red "
            "bell pepper, and a red tuna. I'll make sure my inventory matches.) I",
        "(Good, that's a match. Now I can start following the directions in the "
            "cookbook. Since the recipe begins with slicing, I need to pick up the "
            "knife.) GET KNIFE",
        "(It seems my inventory is full, so I need to drop something to make room for "
            "the knife. Everything I'm holding is a necessary ingredient, but I won't "
            "need the red tuna right away so I can put that down for the time being.) "
            "DROP RAW RED TUNA",
        "(Now I should be able to pick up the knife.) GET KNIFE",
        "(Now I can follow the first step of the recipe, which is to slice the block "
            "of cheese.) SLICE BLOCK OF CHEESE WITH KNIFE",
        "(Next I need to grill the block of cheese. For that I need the BBQ, so I "
            "need to leave the kitchen for the backyard. Fortunately I've already been "
            "there and I remember the way.) E",
        "(I need to continue east again to get to the backyard.) E",
        "(Now I'm in the backyard and can grill the cheese.) "
            "COOK BLOCK OF CHEESE WITH BBQ",
        "(Now that the cheese is grilled, I can return to the kitchen to continue the "
            "recipe.) W",
        "(The kitchen is to the west again.) W",
        "(I'll check the cookbook one more time to remember where I left off.) "
            "READ COOKBOOK",
        "(I just finished grilling the block of cheese, so the next step is to dice "
            "the red bell pepper.) DICE RED BELL PEPPER WITH KNIFE",
        "(Next I need to fry the red bell pepper. I need to remember that frying "
            "uses the stove. If I tried to use a different appliance I'd lose the "
            "game.) COOK RED BELL PEPPER WITH STOVE",
        "(Now I need to dice the red tuna, but I'm not holding it because I put it "
            "down earlier to make room in my inventory for the knife. Since my "
            "inventory is full again and I still need knife, I need to put something "
            "different down to make room for the red tuna. I'm finished now with the "
            "block of cheese and with the red bell pepper, so I can put one of those "
            "down. Let's choose the pepper.) DROP RED BELL PEPPER",
        "(Now I have room to pick up the tuna.) GET RED TUNA",
        "(Now that I'm holding the tuna and the knife, I can dice it according to "
            "the recipe.) DICE RED TUNA WITH KNIFE",
        "(The next step in the recipe is to fry the red tuna. Again, I need to use the "
            "stove to fry anything.) COOK RED TUNA WITH STOVE",
        "(Now I need to prepare the meal. Before I can do that, I need to have all "
            "the ingredients in my inventory, including the red bell pepper which I "
            "set down earlier. Now that I'm finished with the knife, I can drop it.) "
            "DROP KNIFE",
        "(Now I have room to pick up the red bell pepper.) GET RED BELL PEPPER",
        "(Now that I'm holding all the ingredients, I can prepare the meal.) PREPARE "
            "MEAL",
        "(Now I can eat the meal. This should win the game.) EAT MEAL"
    ]
}]

In [27]:
SECOND_EXPERIMENT = ExperimentRunner(
    CLIENTS, MODELS, NEW_INSTRUCTIONS, NEW_SAMPLE_GAMES, TW_MAKE_ARGS, MAX_TURNS
)

In [28]:
SECOND_RESULTS = SECOND_EXPERIMENT.run(
    ['gpt-4o', 'gpt-4o-mini', 'llama3.3-70b-instruct-fp8', 'llama3.1-405b-instruct-fp8'],
    range(101, 201),
    processes=12
)

In [29]:
SECOND_ANALYSIS = Analyzer(SECOND_RESULTS)

In [30]:
SECOND_ANALYSIS.cumulative_outcomes()

{'gpt-4o': [{'won': 94, 'quit': 4, 'turnmax': 2},
  {'won': 97, 'quit': 6, 'turnmax': 3},
  {'won': 98, 'quit': 8, 'turnmax': 3}],
 'gpt-4o-mini': [{'won': 32, 'turnmax': 52, 'quit': 9, 'lost': 7},
  {'won': 40, 'turnmax': 96, 'quit': 13, 'lost': 19},
  {'won': 50, 'turnmax': 139, 'quit': 15, 'lost': 24}],
 'llama3.3-70b-instruct-fp8': [{'turnmax': 22,
   'won': 64,
   'quit': 12,
   'lost': 2},
  {'turnmax': 30, 'won': 84, 'quit': 18, 'lost': 4},
  {'turnmax': 36, 'won': 91, 'quit': 20, 'lost': 5}],
 'llama3.1-405b-instruct-fp8': [{'won': 96, 'turnmax': 1, 'quit': 3},
  {'won': 100, 'turnmax': 1, 'quit': 3},
  {'won': 100, 'turnmax': 1, 'quit': 3}]}

In [31]:
SECOND_ANALYSIS.credible_intervals()

{'gpt-4o': [(np.float64(0.8804934023073464), np.float64(0.9745901000827414)),
  (np.float64(0.9263563355558441), np.float64(0.9922915632034028)),
  (np.float64(0.946468364729441), np.float64(0.9969659481918016))],
 'gpt-4o-mini': [(np.float64(0.23463582261070445),
   np.float64(0.41550577526337756)),
  (np.float64(0.31202237901659136), np.float64(0.5018336444284139)),
  (np.float64(0.4108726603891476), np.float64(0.6039170341734513))],
 'llama3.3-70b-instruct-fp8': [(np.float64(0.5429515076980874),
   np.float64(0.7289977262938208)),
  (np.float64(0.7618373748476049), np.float64(0.903272051823542)),
  (np.float64(0.8480345871691255), np.float64(0.9568176730272397))],
 'llama3.1-405b-instruct-fp8': [(np.float64(0.9076498833353197),
   np.float64(0.9863765591588373)),
  (np.float64(0.9774688951782196), np.float64(0.999995710864218)),
  (np.float64(0.9855870657238253), np.float64(0.999999760222345))]}

The following p-value checks whether the second experiment was a statistically significant improvement on the first.

In [32]:
FIRST_ANALYSIS.test_improvement(SECOND_ANALYSIS)

np.float64(0.006298504998073345)